# Exercice: Modélisation orientée requête (Cassandra)

<table><tr>
    <td><img src="image/esi-sba.png" width="100" height="100"></td>
    <td><img src="image/cassandra.png" width="100" height="100"></td>
    </tr></table>



### Dans ce notebook, nous allons travailler sur la modélisation orientée requête sous la BDD No-SQL Cassandra, en particulier nous traitons les points suivants :

* Dénormaliser une BDD relationnelle en partant du principe 1 table per query 
* Définir la partie Primary Key pour chaque requête
* Optimiser vos modèles en choisissant la bonne Partition Key & Clustering Column
* Valider vos modèles 
* Tester quelques requêtes particulières


#### installer cassandra-driver afin de se connecter au cluster cassandra
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

#### Import Apache Cassandra python package

In [1]:
import cassandra

### créer une connection au cluster cassandra

In [2]:
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) # si cassandra est installé localement avec le port par défaut 9042
    session = cluster.connect()
except Exception as e:
    print(e)

### créer maintenant un keyspace nommé "cart_esi"

In [3]:
try:
  session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cart_esi
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
  )
except Exception as e:
  print(e)


### connecter à votre Keyspace  cart_esi 


In [4]:
try:
    session.set_keyspace('cart_esi')
except Exception as e:
    print(e)

### Supposons qu'on a une plateforme e-commerce dont le   modèle RELATIONNEL est décrit comme suit:

<table><tr>
    <td><img src="image/E-R_model.png" width="500" height="500"></td>
      </tr></table>

### imaginons  que nous souhaitons analyser ces données à  travers 4 requêtes analytiques:
* `Query1= retourner les informations d’un panier (cart_id, state) de chaque utilisateur (user_id). Afficher également l’email de l’utilisateur.`

* `Query2= retourner pour chaque utilisateur toutes les informations de ses paniers ainsi que leurs contenus (les items ajoutés avec leurs quantités). Ordonner le résultat par state(asc) et date de création(asc).`

*  `Query3= retourner pour chaque panier sa date de création, son total et les sous-totaux par item.`

#### Créer les trois tables qui correspondent aux requêtes précédentes, en indiquant pour chacune:
* Primary Key
* Partition Key
* Clustering Column

In [5]:
try:
    
    ### Create the table that corresponds to the Query1
    query1 = "CREATE TABLE IF NOT EXISTS user_cart (user_id int, email varchar, cart_id int, state int, PRIMARY KEY (user_id, cart_id));"
    
    session.execute(query1)
except Exception as e:
        print(e)

In [6]:
try:
    
    ### Create the table that corresponds to the Query2
    query2 = "CREATE TABLE IF NOT EXISTS user_cart_items (user_id int, state int, created_at timestamp, cart_id int, item_id int, item_name varchar, quantity int, PRIMARY KEY (user_id, state, created_at, cart_id, item_id)) WITH CLUSTERING ORDER BY (state ASC, created_at ASC);"
    
    session.execute(query2)
except Exception as e:
        print(e)

In [7]:
try:
    
    ### Create the table that corresponds to the Query3
    query3= "CREATE TABLE cart_totals (cart_id int, created_at timestamp, item_id int, item_name varchar, quantity int, subtotal float, total float, PRIMARY KEY (cart_id, item_id));"
   
    session.execute(query3)
except Exception as e:
        print(e)

### Une fois vous créez les différentes tables, essayez de les peupler.
#### * deux utilisateurs
####  * cinq paniers, trois pour le premier User et deux pour le deuxième
####  * entre 1 et 3 items par panier 



### Insertion des données dans Table1

In [8]:
query = "INSERT INTO user_cart (user_id, email, cart_id, state) "
query = query + " VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, (1, 'user1@example.com', 101, 1))
    session.execute(query, (1, 'user1@example.com', 102, 2))
    session.execute(query, (1, 'user1@example.com', 103, 1))

    session.execute(query, (2, 'user2@example.com', 201, 2))
    session.execute(query, (2, 'user2@example.com', 202, 1))
    
except Exception as e:
        print(e)  


### Insertion des données dans Table2

In [9]:
query = "INSERT INTO user_cart_items (user_id, state, created_at, cart_id, item_id, item_name, quantity) "
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

try:
    # user1 (cart id 101)
    session.execute(query, (1, 1, '2024-12-09 10:00:00', 101, 1, 'Item A', 2))
    session.execute(query, (1, 1, '2024-12-09 10:00:00', 101, 2, 'Item B', 1))

    # user1 (cart id 102)
    session.execute(query, (1, 2, '2024-12-09 10:00:00', 102, 3, 'Item C', 3))

    # user1 (cart id 103)
    session.execute(query, (1, 1, '2024-12-09 10:00:00', 103, 4, 'Item D', 2))

    # user2 (cart id 201)
    session.execute(query, (2, 2, '2024-12-09 10:00:00', 201, 5, 'Item E', 1))
    session.execute(query, (2, 2, '2024-12-09 10:00:00', 201, 6, 'Item F', 2))

    #  user2 (cart id 202)
    session.execute(query, (2, 1, '2024-12-09 10:00:00', 202, 7, 'Item G', 3))
    
except Exception as e:
        print(e)  

### Insertion des données dans Table3

In [10]:
query = "INSERT INTO cart_totals (cart_id, created_at, item_id, item_name, quantity, subtotal, total)  "
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

try:
    # first cart (cart_id 101) for user1
    session.execute(query, (101, '2024-12-09 10:00:00', 1, 'Item A', 2, 20.0, 50.0))
    session.execute(query, (101, '2024-12-09 10:00:00', 2, 'Item B', 1, 15.0, 50.0))

    # second cart (cart_id 102) of user1
    session.execute(query, (102, '2024-12-09 11:00:00', 3, 'Item C', 3, 30.0, 60.0))

    # third cart (cart_id 103) of user1
    session.execute(query, (103, '2024-12-09 12:00:00', 4, 'Item D', 2, 25.0, 50.0))

    # first cart (cart_id 201) of user2
    session.execute(query, (201, '2024-12-09 13:00:00', 5, 'Item E', 1, 20.0, 40.0))
    session.execute(query, (201, '2024-12-09 13:00:00', 6, 'Item F', 2, 30.0, 40.0))

    # second cart (cart_id 202) of user2
    session.execute(query, (202, '2024-12-09 14:00:00', 7, 'Item G', 3, 45.0, 50.0))
    
except Exception as e:
        print(e)  

### Valider votre modèle en exécutant les requêtes suivantes

* Retourner les paniers de l'utilisateur N°1"

* Retourner le contenu des paniers "saved" de l'utilisateur N°1

* retourner le total et les soustotaux du panier N°3

In [11]:
query = "SELECT * FROM user_cart WHERE user_id = 1;"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)       

Row(user_id=1, cart_id=101, email='user1@example.com', state=1)
Row(user_id=1, cart_id=102, email='user1@example.com', state=2)
Row(user_id=1, cart_id=103, email='user1@example.com', state=1)


In [12]:
query = "SELECT * FROM user_cart_items WHERE user_id = 1 AND state = 1;"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)    

Row(user_id=1, state=1, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=101, item_id=1, item_name='Item A', quantity=2)
Row(user_id=1, state=1, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=101, item_id=2, item_name='Item B', quantity=1)
Row(user_id=1, state=1, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=103, item_id=4, item_name='Item D', quantity=2)


In [13]:
query = "SELECT total, item_id, item_name, quantity, subtotal FROM cart_totals WHERE cart_id = 103;"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)         

Row(total=50.0, item_id=4, item_name='Item D', quantity=2, subtotal=25.0)


### Essayons d'autres requêtes 

* `Retourner le contenu des paniers créés  aprés le 01-12-2024 par l'utilisateur N°1` 

In [15]:
query = "SELECT * FROM user_cart_items WHERE user_id = 1 AND created_at > '2024-12-01' ALLOW FILTERING;"

try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)      

Row(user_id=1, state=1, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=101, item_id=1, item_name='Item A', quantity=2)
Row(user_id=1, state=1, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=101, item_id=2, item_name='Item B', quantity=1)
Row(user_id=1, state=1, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=103, item_id=4, item_name='Item D', quantity=2)
Row(user_id=1, state=2, created_at=datetime.datetime(2024, 12, 9, 10, 0), cart_id=102, item_id=3, item_name='Item C', quantity=3)


### Proposer une solution pour optimiser cette requête

In [17]:
from datetime import datetime
### make your solution 
## table creation
### Create the table that corresponds to the Query4
query2 = "CREATE TABLE IF NOT EXISTS user_cart_items_2 (user_id int, cart_id int, state int, created_at timestamp, PRIMARY KEY (user_id, created_at, cart_id));"

   
session.execute(query2) 

## insert data
query = "INSERT INTO user_cart_items_2 (user_id, cart_id, state, created_at)"
query = query + " VALUES (%s, %s, %s, %s)"

data = [
    (1, 101, 1, datetime(2024, 12, 1, 10, 30)),  
    (1, 102, 1, datetime(2024, 12, 2, 15, 0)),   
    (2, 201, 0, datetime(2024, 12, 3, 11, 45)),  
    (2, 202, 1, datetime(2024, 12, 4, 9, 30))    
]

try:
    for record in data:
        session.execute(query, record)
   
  
except Exception as e :
        print(e)   

In [19]:
# query4
query = "SELECT * FROM user_cart_items_2 WHERE user_id = 1 AND created_at > '2024-12-01';"


try:
    rows = session.execute(query)
    for row in rows:
          print (row)
except Exception as e:
        print(e)  

Row(user_id=1, created_at=datetime.datetime(2024, 12, 1, 10, 30), cart_id=101, state=1)
Row(user_id=1, created_at=datetime.datetime(2024, 12, 2, 15, 0), cart_id=102, state=1)


###  drop  tables. 

### close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()